In [48]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import pandas as pd


In [ ]:
chrome_options = webdriver.ChromeOptions()

Configurando o webdriver

In [ ]:
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
wd_chrome = webdriver.Chrome(chrome_options=chrome_options)

Carregando a page e referenciando o botão com o xpath

In [ ]:
wd_chrome.get('https://covid.saude.gov.br/')

In [ ]:
csv_button = wd_chrome.find_element_by_xpath('/html/body/app-root/ion-app/ion-router-outlet/app-home/ion-content/div[1]/div[2]/ion-button')

In [40]:
csv_button.click()

Descompactando o arquivo rar

In [41]:
import rarfile

r = rarfile.RarFile('HIST_PAINEL_COVIDBR_08ago2021.rar')
r.extractall()
r.close()

In [43]:
rarfile.UNRAR_TOOL = "unrar"

In [72]:
cols = ['regiao', 'estado', 'municipio', 'coduf', 'codmun', 'codRegiaoSaude',
       'nomeRegiaoSaude', 'data', 'semanaEpi', 'populacaoTCU2019', 'casosNovos', 'obitosAcumulado', 'obitosNovos',
       'Recuperadosnovos', 'emAcompanhamentoNovos', 'interior/metropolitana']

In [73]:
df = pd.read_csv(r'HIST_PAINEL_COVIDBR_2020_Parte1_08ago2021.csv', sep = ';',usecols= cols)

In [74]:
df.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-25,9,210147125.0,0,0,0,NaN,NaN,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-26,9,210147125.0,1,0,0,NaN,NaN,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-27,9,210147125.0,0,0,0,NaN,NaN,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-28,9,210147125.0,0,0,0,NaN,NaN,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-29,9,210147125.0,1,0,0,NaN,NaN,NaN


In [ ]:
df.estado.value_counts

#Preprocessing data

In [90]:
data = df.loc[df['estado']== 'BA']

In [95]:
data.shape

(53244, 16)

In [93]:
data.isnull().sum()

regiao                        0
estado                        0
municipio                   285
coduf                         0
codmun                      158
codRegiaoSaude              285
nomeRegiaoSaude             285
data                          0
semanaEpi                     0
populacaoTCU2019            127
casosNovos                    0
obitosAcumulado               0
obitosNovos                   0
Recuperadosnovos          53244
emAcompanhamentoNovos     53244
interior/metropolitana      285
dtype: int64

In [98]:
data = data.drop(columns=['Recuperadosnovos', 'emAcompanhamentoNovos'])

In [99]:
data.fillna("0",inplace=True)

In [100]:
data.isnull().sum()

regiao                    0
estado                    0
municipio                 0
coduf                     0
codmun                    0
codRegiaoSaude            0
nomeRegiaoSaude           0
data                      0
semanaEpi                 0
populacaoTCU2019          0
casosNovos                0
obitosAcumulado           0
obitosNovos               0
interior/metropolitana    0
dtype: int64

In [103]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53244 entries, 2528 to 290935
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   regiao                  53244 non-null  object
 1   estado                  53244 non-null  object
 2   municipio               53244 non-null  object
 3   coduf                   53244 non-null  int64 
 4   codmun                  53244 non-null  object
 5   codRegiaoSaude          53244 non-null  object
 6   nomeRegiaoSaude         53244 non-null  object
 7   data                    53244 non-null  object
 8   semanaEpi               53244 non-null  int64 
 9   populacaoTCU2019        53244 non-null  object
 10  casosNovos              53244 non-null  int64 
 11  obitosAcumulado         53244 non-null  int64 
 12  obitosNovos             53244 non-null  int64 
 13  interior/metropolitana  53244 non-null  object
dtypes: int64(5), object(9)
memory usage: 6.1+ MB


In [104]:
data.columns

Index(['regiao', 'estado', 'municipio', 'coduf', 'codmun', 'codRegiaoSaude',
       'nomeRegiaoSaude', 'data', 'semanaEpi', 'populacaoTCU2019',
       'casosNovos', 'obitosAcumulado', 'obitosNovos',
       'interior/metropolitana'],
      dtype='object')

In [101]:
data.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosNovos,obitosAcumulado,obitosNovos,interior/metropolitana
2528,Nordeste,BA,0,29,0,0,0,2020-02-25,9,1.48731e+07,0,0,0,0
2529,Nordeste,BA,0,29,0,0,0,2020-02-26,9,1.48731e+07,0,0,0,0
2530,Nordeste,BA,0,29,0,0,0,2020-02-27,9,1.48731e+07,0,0,0,0
2531,Nordeste,BA,0,29,0,0,0,2020-02-28,9,1.48731e+07,0,0,0,0
2532,Nordeste,BA,0,29,0,0,0,2020-02-29,9,1.48731e+07,0,0,0,0


In [109]:
datat = data.loc[(pd.to_numeric(data.obitosNovos) >= 0) & (pd.to_numeric(data.casosNovos) >= 0)]

#Data visualization

In [80]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [110]:
fig = px.bar(datat, x='data', y=datat['casosNovos'],title= 'Cases per day Bahia ', 
             labels={ "data": "Days", "casosNovos": "New Cases"})
fig.show()

In [111]:
fig = px.bar(datat, x='data', y=datat['obitosNovos'],title= 'Deaths per day Bahia ', 
             labels={ "data": "Days", "casosNovos": "New Cases"})
fig.show()